# OpenAI Embeddings Tutorial

이 노트북에서는 LangChain의 OpenAIEmbeddings를 사용하여 텍스트를 벡터로 변환하고 유사도를 계산하는 방법을 학습합니다.

## 학습 목표
- OpenAI의 임베딩 모델 이해
- 텍스트를 벡터로 변환하는 방법
- 임베딩 차원 조절
- 코사인 유사도를 통한 문장 간 유사도 계산

## 1. 문서 임베딩 개요

문서 임베딩은 문서의 내용을 수치적인 벡터로 변환하는 과정입니다. 

이 과정을 통해 문서의 의미를 수치화하고, 다양한 자연어 처리 작업에 활용할 수 있습니다. 대표적인 사전 학습된 언어 모델로는 BERT와 GPT가 있으며, 이러한 모델들은 문맥적 정보를 포착하여 문서의 의미를 인코딩합니다. 

문서 임베딩은 토큰화된 문서를 모델에 입력하여 임베딩 벡터를 생성하고, 이를 평균하여 전체 문서의 벡터를 생성합니다. 이 벡터는 문서 분류, 감성 분석, 문서 간 유사도 계산 등에 활용될 수 있습니다.

[OpenAI Embeddings 공식 문서](https://platform.openai.com/docs/guides/embeddings/embedding-models)

## 2. OpenAI 임베딩 모델 비교

| MODEL                  | PAGES PER DOLLAR | PERFORMANCE ON MTEB EVAL | MAX INPUT |
|------------------------|------------------|---------------------------|-----------|
| text-embedding-3-small | 62,500           | 62.3%                     | 8191      |
| text-embedding-3-large | 9,615            | 64.6%                     | 8191      |
| text-embedding-ada-002 | 12,500           | 61.0%                     | 8191      |

- **text-embedding-3-small**: 비용 효율적이며 대부분의 사용 사례에 적합
- **text-embedding-3-large**: 더 높은 성능이 필요한 경우 사용
- **text-embedding-ada-002**: 이전 세대 모델

## 3. 환경 설정

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 4. OpenAIEmbeddings 기본 사용법

In [2]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
# 테스트용 샘플 텍스트
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

## 5. 쿼리 임베딩

`embeddings.embed_query(text)`는 주어진 텍스트를 임베딩 벡터로 변환하는 함수입니다.

이 함수는 텍스트를 벡터 공간에 매핑하여 의미적으로 유사한 텍스트를 찾거나 텍스트 간의 유사도를 계산하는 데 사용될 수 있습니다.

**사용 시나리오:**
- 검색 쿼리 벡터화
- 단일 텍스트의 임베딩이 필요한 경우

In [4]:
# 텍스트를 임베딩하여 쿼리 결과를 생성합니다.
query_result = embeddings.embed_query(text)

In [5]:
# 쿼리 결과의 처음 5개 항목을 확인합니다.
query_result[:5]

[-0.00776276458054781,
 0.03680367395281792,
 0.019545823335647583,
 -0.0196656696498394,
 0.017203375697135925]

In [6]:
# 기본 임베딩 차원 확인
print(f"기본 임베딩 차원: {len(query_result)}")

기본 임베딩 차원: 1536


## 6. Document 임베딩

`embeddings.embed_documents()` 함수를 사용하여 여러 텍스트 문서를 한 번에 임베딩합니다.

**특징:**
- 리스트 형태로 여러 문서를 한 번에 처리
- 배치 처리로 효율적인 벡터화
- 문서 검색 시스템 구축에 활용

In [7]:
# 여러 문서를 한 번에 임베딩
doc_result = embeddings.embed_documents(
    [text, text, text, text]
)  # 동일한 텍스트를 4번 임베딩

In [8]:
# 임베딩된 문서의 개수 확인
len(doc_result)

4

In [9]:
# 첫 번째 문서의 임베딩 벡터 일부 확인
doc_result[0][:5]

[-0.00776276458054781,
 0.03680367395281792,
 0.019545823335647583,
 -0.0196656696498394,
 0.017203375697135925]

In [10]:
# 동일한 텍스트의 쿼리 임베딩과 문서 임베딩 비교
print(f"쿼리 임베딩과 문서 임베딩이 동일한가요? {query_result == doc_result[0]}")

쿼리 임베딩과 문서 임베딩이 동일한가요? True


## 7. 임베딩 차원 조절

`text-embedding-3` 모델 클래스를 사용하면 반환되는 임베딩의 크기를 지정할 수 있습니다.

**차원 조절의 장점:**
- 저장 공간 절약
- 계산 속도 향상
- 특정 작업에 최적화된 차원 선택 가능

**주의사항:**
- 차원을 줄이면 정보 손실이 발생할 수 있음
- 작업의 복잡도에 따라 적절한 차원 선택 필요

In [11]:
# 1024차원의 임베딩을 생성하는 객체를 초기화합니다.
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [12]:
# 차원이 1024로 설정되었는지 확인
len(embeddings_1024.embed_documents([text])[0])

1024

In [13]:
# 다양한 차원으로 임베딩 생성 비교
embeddings_512 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)
embeddings_1536 = OpenAIEmbeddings(model="text-embedding-3-small")  # 기본값

print(f"512차원 임베딩 크기: {len(embeddings_512.embed_query(text))}")
print(f"1024차원 임베딩 크기: {len(embeddings_1024.embed_query(text))}")
print(f"1536차원 임베딩 크기(기본): {len(embeddings_1536.embed_query(text))}")

512차원 임베딩 크기: 512
1024차원 임베딩 크기: 1024
1536차원 임베딩 크기(기본): 1536


## 8. 유사도 계산

임베딩 벡터를 사용하여 텍스트 간의 의미적 유사도를 계산할 수 있습니다.
코사인 유사도는 두 벡터 간의 각도를 측정하여 -1에서 1 사이의 값을 반환합니다.

**코사인 유사도 해석:**
- 1에 가까울수록: 매우 유사함
- 0에 가까울수록: 관련성이 낮음
- -1에 가까울수록: 반대 의미 (실제로는 거의 발생하지 않음)

In [14]:
# 유사도 테스트를 위한 다양한 문장들
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"  # 구두점만 다름
sentence3 = "안녕하세요? 만나서 반가워요."  # 유사한 의미
sentence4 = "Hi, nice to meet you."  # 영어로 같은 의미
sentence5 = "I like to eat apples."  # 전혀 다른 주제

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

# 모든 문장을 임베딩
sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [16]:
# 코사인 유사도 계산 함수
def similarity(a, b):
    """두 임베딩 벡터 간의 코사인 유사도를 계산합니다."""
    return cosine_similarity([a], [b])[0][0]

In [17]:
# 모든 문장 쌍의 유사도 계산 및 출력
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:  # 중복 계산 방지
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9644] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다!
[유사도 0.8376] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.5042] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1362] 안녕하세요? 반갑습니다. 	 <=====> 	 I like to eat apples.
[유사도 0.8142] 안녕하세요? 반갑습니다! 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.4790] 안녕하세요? 반갑습니다! 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1318] 안녕하세요? 반갑습니다! 	 <=====> 	 I like to eat apples.
[유사도 0.5127] 안녕하세요? 만나서 반가워요. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1409] 안녕하세요? 만나서 반가워요. 	 <=====> 	 I like to eat apples.
[유사도 0.2250] Hi, nice to meet you. 	 <=====> 	 I like to eat apples.


## 9. 유사도 분석

위 결과를 분석해보면:

1. **구두점만 다른 경우 (0.9644)**: 거의 동일한 의미로 인식
2. **같은 언어, 유사한 표현 (0.8376, 0.8142)**: 높은 유사도
3. **다른 언어, 같은 의미 (0.5042, 0.4790, 0.5127)**: 중간 정도의 유사도
4. **완전히 다른 주제 (0.1362, 0.1318, 0.1409)**: 낮은 유사도

이를 통해 OpenAI 임베딩이 의미론적 유사성을 잘 포착함을 확인할 수 있습니다.

## 10. 실용적인 예제: 간단한 문서 검색 시스템

In [18]:
# 간단한 문서 데이터베이스
documents = [
    "인공지능과 머신러닝은 현대 기술의 핵심입니다. 딥러닝은 머신러닝의 한 분야입니다.",
    "파이썬은 데이터 사이언스와 머신러닝에 가장 인기 있는 프로그래밍 언어입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 처리하는 기술입니다.",
    "클라우드 컴퓨팅은 인터넷을 통해 컴퓨팅 자원을 제공하는 서비스입니다.",
    "데이터 분석은 비즈니스 의사결정에 중요한 역할을 합니다."
]

# 문서 임베딩
doc_embeddings = embeddings_1024.embed_documents(documents)

# 검색 쿼리
query = "머신러닝을 배우고 싶어요"
query_embedding = embeddings_1024.embed_query(query)

# 유사도 계산 및 정렬
similarities = []
for i, doc_embedding in enumerate(doc_embeddings):
    sim = similarity(query_embedding, doc_embedding)
    similarities.append((i, sim, documents[i]))

# 유사도 기준으로 정렬
similarities.sort(key=lambda x: x[1], reverse=True)

# 상위 3개 결과 출력
print(f"\n쿼리: '{query}'\n")
print("검색 결과:")
for i, (idx, sim, doc) in enumerate(similarities[:3]):
    print(f"{i+1}. [유사도: {sim:.4f}] {doc}")


쿼리: '머신러닝을 배우고 싶어요'

검색 결과:
1. [유사도: 0.7893] 인공지능과 머신러닝은 현대 기술의 핵심입니다. 딥러닝은 머신러닝의 한 분야입니다.
2. [유사도: 0.6543] 파이썬은 데이터 사이언스와 머신러닝에 가장 인기 있는 프로그래밍 언어입니다.
3. [유사도: 0.4012] 데이터 분석은 비즈니스 의사결정에 중요한 역할을 합니다.


## 11. 임베딩 사용 시 주의사항 및 모범 사례

### 주의사항
1. **API 비용**: 대량의 텍스트를 임베딩할 때는 비용을 고려해야 합니다.
2. **토큰 제한**: 최대 8191 토큰까지 처리 가능합니다.
3. **차원 선택**: 용도에 맞는 적절한 차원을 선택하세요.

### 모범 사례
1. **배치 처리**: `embed_documents()`를 사용하여 여러 문서를 한 번에 처리
2. **캐싱**: 자주 사용하는 임베딩은 저장하여 재사용
3. **정규화**: 필요에 따라 텍스트 전처리 수행
4. **모델 선택**: 성능과 비용의 균형을 고려하여 모델 선택

## 12. 요약

이 튜토리얼에서 학습한 내용:

1. **OpenAI 임베딩 모델**의 종류와 특징
2. **쿼리 임베딩**과 **문서 임베딩**의 차이점과 사용법
3. **임베딩 차원 조절** 방법
4. **코사인 유사도**를 사용한 텍스트 유사도 계산
5. 간단한 **문서 검색 시스템** 구현

OpenAI 임베딩은 다양한 NLP 작업의 기초가 되는 중요한 기술입니다. RAG(Retrieval-Augmented Generation), 시맨틱 검색, 문서 분류 등 다양한 응용 분야에서 활용할 수 있습니다.